In [ ]:
import tensorflow as tf

# Download the mnist dataset using keras
data_train, data_test = tf.keras.datasets.mnist.load_data()

# Parse images and labels
(train_images, train_labels) = data_train
(test_images, test_labels) = data_test

# Numpy reshape & type casting
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')


# Normalizing the images to the range of [0., 1.]
train_images /= 255.
test_images /= 255.

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [ ]:
from tensorflow.keras import Model
# Construct a tf.keras.model using tf.keras
class MyCNN(Model):
  def __init__(self):
    super(MyCNN, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='valid')
    self.conv2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='valid')
    self.conv3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='valid')
    self.maxpool = tf.keras.layers.MaxPooling2D((2, 2))
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(256, activation='relu')
    self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.maxpool(x)

    x = self.conv2(x)
    x = self.maxpool(x)

    x = self.conv3(x)
    x = self.maxpool(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.dense2(x)
    
    return x

# Create model
model = MyCNN()

In [ ]:
# Choose loss function and optimizer for training
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Metrics to measure loss and accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
# Define function for training
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# Define function for testing
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  loss = loss_fn(labels, predictions)

  test_loss(loss)
  test_accuracy(labels, predictions)

In [ ]:
batch_size = 128

# Prepare the dataset using tf.data
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.batch(batch_size)



EPOCHS = 10

for epoch in range(EPOCHS):
    # Reset the metrics at each epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
      train_step(images, labels)

    for images, labels in test_ds:
      test_step(images, labels)

    print('Epoch: %02d' % (epoch + 1),
          'Loss = {:2.4f}'.format(train_loss.result()),
          'Train accuracy = {:2.4f}'.format(train_accuracy.result()),
          'Test loss = {:2.4f}'.format(test_loss.result()),
          'Test accuracy = {:2.4f}'.format(test_accuracy.result()))

Epoch: 01 Loss = 0.3077 Train accuracy = 0.9059 Test loss = 0.0938 Test accuracy = 0.9708
Epoch: 02 Loss = 0.0897 Train accuracy = 0.9728 Test loss = 0.0639 Test accuracy = 0.9812
Epoch: 03 Loss = 0.0611 Train accuracy = 0.9807 Test loss = 0.0638 Test accuracy = 0.9807
Epoch: 04 Loss = 0.0473 Train accuracy = 0.9853 Test loss = 0.0585 Test accuracy = 0.9819
Epoch: 05 Loss = 0.0403 Train accuracy = 0.9873 Test loss = 0.0579 Test accuracy = 0.9831
Epoch: 06 Loss = 0.0328 Train accuracy = 0.9894 Test loss = 0.0490 Test accuracy = 0.9862
Epoch: 07 Loss = 0.0267 Train accuracy = 0.9916 Test loss = 0.0467 Test accuracy = 0.9863
Epoch: 08 Loss = 0.0222 Train accuracy = 0.9930 Test loss = 0.0639 Test accuracy = 0.9833
Epoch: 09 Loss = 0.0205 Train accuracy = 0.9933 Test loss = 0.0481 Test accuracy = 0.9885
Epoch: 10 Loss = 0.0163 Train accuracy = 0.9945 Test loss = 0.0504 Test accuracy = 0.9867
